In [1]:
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from typing import TypedDict

# 1. Define the state to pass through nodes
class ReasonedAgentState(TypedDict):
    query: str
    analysis: str
    answer: str

# 2. Initialize the LLM
llm = ChatOpenAI(model="gpt-3.5-turbo")

# 3. Reflection/Analysis Node
def analyze_node(state):
    query = state['query']
    system_prompt = (
        "You are an expert agent. Before taking any actions, analyze the request. "
        "Break it down step by step. Clearly state your reasoning. "
        "After reasoning, outline the best steps to answer the user's query."
    )
    chain = lambda q: llm([SystemMessage(content=system_prompt), HumanMessage(content=q)]).content
    analysis = chain(query)
    state["analysis"] = analysis
    return state

# 4. Execution Node
def execute_node(state):
    system_prompt = (
        "You have just performed the following analysis and broken down the steps:\n"
        f"{state['analysis']}\n"
        "Now, follow those steps and answer the user's query."
    )
    query = state['query']
    chain = lambda q: llm([SystemMessage(content=system_prompt), HumanMessage(content=query)]).content
    answer = chain(query)
    state["answer"] = answer
    return state

# 5. Build the graph
graph_builder = StateGraph(ReasonedAgentState)
graph_builder.add_node("analyze", analyze_node)
graph_builder.add_node("execute", execute_node)
graph_builder.set_entry_point("analyze")
graph_builder.add_edge("analyze", "execute")
graph_builder.add_edge("execute", END)

graph = graph_builder.compile()

# 6. Test it!
initial_state = ReasonedAgentState(query="How can I backup my photos from my phone to my laptop?")
final_state = graph.invoke(initial_state)
print("Analysis / Steps:\n", final_state['analysis'])
print("\nFinal Answer:\n", final_state['answer'])

C:\Users\johnh\AppData\Local\Temp\ipykernel_54236\791651496.py:23: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain = lambda q: llm([SystemMessage(content=system_prompt), HumanMessage(content=q)]).content


Analysis / Steps:
 To backup photos from a phone to a laptop, several methods can be used depending on your specific devices and preferences. Here are the steps to accomplish this task:

1. **Connect Phone to Laptop**: First, ensure your phone and laptop are both turned on and have their respective USB cables. 
   - Connect your phone to your laptop using the USB cable. Ensure the connection is secure.

2. **Transfer Photos via USB Cable**:
   - Once connected, your laptop should detect your phone as a storage device.
   - Open the file explorer on your laptop and locate your phone's storage. Look for the DCIM folder, which usually contains your photos and videos.
   - Copy and paste the desired photos from your phone to a folder on your laptop to backup them.

3. **Use Photo Management Software**:
   - You can also use software like Google Photos, iCloud, or third-party apps to automate the backup process.
   - Install any required software on both devices and follow the setup instruc